In [79]:
import os
import requests
import random
KEY = '91mTqXFEeSaCIbjC0fuaB1RABk4HANqS4I91qRBN'
KEY2 = 'flLQZGa0kx8opciUA6M7R9iLPwfFPj5yFKKlCiVb'
MOST_RECENT_HR_BILL = 1490
MOST_RECENT_S_BILL = 606
CONGRESS = 115

In [89]:
def getJSON(webpage):
    headers = {
        'X-API-Key': KEY
    }
    r = requests.get(webpage, headers=headers)
    return r.json()

def getSponsors(bill_id, congress):
    '''Returns (PrimarySponsor, [cosponsors]) for specified bill_id and congress strings. Returns () if faulty'''
    edge = getJSON('https://api.propublica.org/congress/v1/' + str(congress) + '/bills/' + bill_id + '/cosponsors.json')
    if edge['status'] == 'OK':
        return (edge['results'][0]['sponsor_id'], [cosponsor['cosponsor_id'] for cosponsor in edge['results'][0]['cosponsors']])
    else:
        return ()
    
def getMembers(congress):
    '''takes congress session and Returns dictioanry of congress member ids with value 0'''
    senate_members = getJSON('https://api.propublica.org/congress/v1/'+ str(congress) +'/senate/members.json')
    house_members = getJSON('https://api.propublica.org/congress/v1/'+ str(congress) +'/house/members.json')
    senateList, houseList = [],[]
    if senate_members['status'] == 'OK':
        senateList = [(senator['id'],0) for senator in senate_members['results'][0]['members']]
        print('senate members', len(senateList))
    if house_members['status'] == 'OK':
        houseList = [(rep['id'],0) for rep in house_members['results'][0]['members']]
        print('house members', len(houseList))
    return dict(senateList + houseList)
    
    
def buildGraph(members, congress):
    graph = {}
    for m in members:
        graph[m] = []
    
    
    for i in range(1, MOST_RECENT_HR_BILL+1):
        print("At HR{0}".format(i))
        edge = getSponsors("hr{0}".format(i), congress)
        if edge: # not (,)
            graph[edge[0]].extend(edge[1])
    
    for i in range(1, MOST_RECENT_S_BILL+1):
        print("At S{0}".format(i))
        edge = getSponsors("s{0}".format(i), congress)
        if edge: # not (,)
            graph[edge[0]].extend(edge[1])
        
    return graph
    
    
    

In [91]:
members = getMembers(CONGRESS)


senate members 101
house members 441


In [92]:
graph = buildGraph(members, CONGRESS)

At HR1
At HR2
At HR3
At HR4
At HR5
At HR6
At HR7
At HR8
At HR9
At HR10
At HR11
At HR12
At HR13
At HR14
At HR15
At HR16
At HR17
At HR18
At HR19
At HR20
At HR21
At HR22
At HR23
At HR24
At HR25
At HR26
At HR27
At HR28
At HR29
At HR30
At HR31
At HR32
At HR33
At HR34
At HR35
At HR36
At HR37
At HR38
At HR39
At HR40
At HR41
At HR42
At HR43
At HR44
At HR45
At HR46
At HR47
At HR48
At HR49
At HR50
At HR51
At HR52
At HR53
At HR54
At HR55
At HR56
At HR57
At HR58
At HR59
At HR60
At HR61
At HR62
At HR63
At HR64
At HR65
At HR66
At HR67
At HR68
At HR69
At HR70
At HR71
At HR72
At HR73


KeyboardInterrupt: 

In [64]:
def ranker(members, graph, p=0.5, iterations=100000):
    assert len(graph) == len(members)
    current = random.choice(graph.keys())
    members[current] += 1
    for _ in range(iterations):
        if len(graph[current]) == 0 or random() < p:
            current = random.choice(graph.keys())
        else:
            neighbors = graph[current]
            current = neighbors[random.randint(0, len(neighbors)-1)]
        members[current] += 1

SyntaxError: unexpected EOF while parsing (<ipython-input-64-c5f9934ec014>, line 17)

In [ ]:
ranker()
member_ranks = [pair[0] for pair in sorted(members.items(), key = lambda item: item[1], reverse=True)]
top_score = members[member_ranks[0]]
for i in range(len(member_ranks)):
    print "%d %f: %s" % (i+1,members[member_ranks[i]]/top_score, member_ranks[i])